<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Практика" data-toc-modified-id="Практика-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Практика</a></span><ul class="toc-item"><li><span><a href="#Загрузка-DataFrame" data-toc-modified-id="Загрузка-DataFrame-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Загрузка DataFrame</a></span><ul class="toc-item"><li><span><a href="#Задача-1" data-toc-modified-id="Задача-1-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Задача 1</a></span></li></ul></li><li><span><a href="#Теория-вероятности.-События" data-toc-modified-id="Теория-вероятности.-События-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Теория вероятности. События</a></span><ul class="toc-item"><li><span><a href="#Задача-2" data-toc-modified-id="Задача-2-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Задача 2</a></span></li><li><span><a href="#Задача-3" data-toc-modified-id="Задача-3-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Задача 3</a></span></li></ul></li></ul></li></ul></div>

# Практика
Используемые библиотеки

In [1]:
import pandas as pd
import numpy as np
import requests
import io

## Загрузка DataFrame
### Задача 1
На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad
средствами библиотеки __Pandas__ сформируйте поле выводов по каждому анализирумомому параметру.
Например, по показателю _pH_ получен результат _8.4 единицы pH_ при нормативе от _6 до 9 единиц pH_. Т.о. по данному показателю результат анализа в норме.
Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, __6 >= pH >= 9__.
В итоговом DataFrame столбец "Показатель" сделайте индексным.


Загзрузка DataFrame выполняется непосредственно c сайта "Открытые данные России" https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (см. код ниже).


In [2]:
url ="https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv"
s = requests.get(url).content
df=pd.read_csv(io.StringIO(s.decode('UTF8')), index_col=['Показатель'])
display(df)

,Единица измерений,Результат анализа,Норматив
Показатель,,,
pH,единицы pH,8.4,в пределах 6-9
Запах,баллы,1,не более 2-3
Цветность,градусы,б/цвета,не более 30
Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
Нитраты (по NO3),мг/дм3,24,не более 45
Фосфаты (P),мг/дм3,0.36,"не более 3,5"
Хлориды (Cl),мг/дм3,200,не более 350


In [3]:
def low_range(data):
    data = data.replace(',', '.')
    if data.startswith('в пределах '):
        data = data.strip('в пределах ')
        try:
            min_value, max_value = map(float, data.split('-'))
            return min_value
        except Exception:
            return "unknown"
    elif data.startswith('не более '):
        return 0
    else:
        return "unknown"
    
def high_range(data):
    data = data.replace(',', '.')
    if data.startswith('в пределах '):
        data = data.strip('в пределах ')
        try:
            min_value, max_value = map(float, data.split('-'))
            return max_value
        except Exception:
            return "unknown"
    elif data.startswith('не более '):
        try:
            result = float(data.strip('не более '))
            return result
        except Exception:
            return "unknown"

def analyzed_convert(data):
    data = data.replace(',', '.')
    try:
        return float(data)
    except Exception:
        return 0
        
# df.drop("Норматив", axis=1, inplace=True)
df["Норматив_min"] = df["Норматив"].apply(low_range)
df["Норматив_max"] = df["Норматив"].apply(high_range)

# одно значение осталось незаполненным - заполняем
df.at['Запах', 'Норматив_max'] = 2.0

df["Результат анализа"] = df["Результат анализа"].apply(analyzed_convert)
df["Соотвествует нормативу"] = ((df["Норматив_min"] <= df["Результат анализа"]) &
                                (df["Результат анализа"] <= df["Норматив_max"]))
df

,Единица измерений,Результат анализа,Норматив,Норматив_min,Норматив_max,Соотвествует нормативу
Показатель,,,,,,
pH,единицы pH,8.400,в пределах 6-9,6.0,9.0,True
Запах,баллы,1.000,не более 2-3,0.0,2.0,True
Цветность,градусы,0.000,не более 30,0.0,30.0,True
Жёсткость,мг-эквл/дм3,9.200,в пределах 7-10,7.0,10.0,True
Аммиак и аммоний-ион (по азоту),мг/дм3,0.420,"не более 1,5",0.0,1.5,True
Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",0.0,3.3,True
Нитраты (по NO3),мг/дм3,24.000,не более 45,0.0,45.0,True
Фосфаты (P),мг/дм3,0.360,"не более 3,5",0.0,3.5,True
Хлориды (Cl),мг/дм3,200.000,не более 350,0.0,350.0,True


## Теория вероятности. События

Требуется сгенерировать необходимые выборки и произвести по ним расчеты

### Задача 2
В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?

В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?


In [4]:
%%time
def sample_check(sampling):
    sample = np.random.choice(sampling, size=3)
    return 'яблоко' not in sample

box = np.array(['яблоко'] * 4 + ['апельсин'] * 5)
number_of_tests = 10**5
result = sum(sample_check(box) for _ in range(number_of_tests))
print(f"Количество испытаний: {number_of_tests}.\nЭкспериментальная верояноcть: {result/number_of_tests}")

Количество испытаний: 100000.
Экспериментальная верояноcть: 0.17311
Wall time: 5 s


- Обнаружена экспериментальная вероятность со значением 0.17

### Задача 3
Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?


В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

In [5]:
%%time
def detail_check(sampling):
    sample = np.random.choice(sampling, size=2)
    return sample[0] == 'bad' and sample[1] == 'good'

details = np.array(['good'] * 7 + ['bad'] * 3)
number_of_tests = 10**5
result = sum(detail_check(details) for _ in range(number_of_tests))
print(f"Количество испытаний: {number_of_tests}.\nЭкспериментальная верояноcть: {result/number_of_tests}")

Количество испытаний: 100000.
Экспериментальная верояноcть: 0.20815
Wall time: 4.24 s


- Обнаружена экспериментальная вероятность со значением 0.21